In [1]:
#!pip install sklearn
#!pip install xgboost
#!pip install keras

In [2]:
import ISee
import Profiler

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

import numpy as np

import Code_vin
import NullHypothesis

import re #REGEX sera utilizado para implementar as heuristicas de pre-processamento

Using TensorFlow backend.


In [3]:
def multisub(patterns, strns, text_pos):
    for strn in strns:
        for pattern, subst in patterns:
            strn[text_pos] = re.sub(pattern, subst, strn[text_pos])
    return strns

In [4]:
#A: bigrams
#B: all characters to lowercase
#C: accentuation removal
#D: special character treatment
#E: stop-words removal
#F: twitter user names removal
#G: twitter topics removal
#H: Reduction of laugh expressions to a common token
laugh_token = ' _laugh_ '
heuristics = {
    'A': [('not( [^ ]+)', 'not_\1')],
    'B': [('([A-Z]*)', lambda x: x.group(0).lower())],
    'C': [('á', 'a'), ('é', 'e'), ('í', 'i'), ('ó', 'o'), ('ú', 'u'), ('à', 'a'), ('ã', 'a'), ('ẽ', 'e'), ('ĩ', 'i'), ('õ', 'o'), ('ũ', 'u'),  ('â', 'a'), ('ê', 'e'), ('ñ', 'n')],
    'D': [(':\)', ''), (':\(', ''), (':\^\)', ''), (':p', ''), (':3', ''), (':c', ''), ('c:', ''), (':o', '')],
    'E': [(' the ', ' '), (' a ', ' '), (' e ', ' '), (' o ', ' ')],
    'F': [('@[^ ]+', ''), ('@[^ ]+ ', '')],
    'G': [('#[^ ]+', ''), ('#[^ ]+ ', '')],
    'H': [(' [ashu]{3,} ', laugh_token), (' [ah]{3,} ', laugh_token), (' [eh]{3,} ', laugh_token), (' k{4,} ', laugh_token), (' [rs]{3,} ', laugh_token), (' [ehu]{3,} ', laugh_token)],
}

In [5]:
#Configuracoes
heuristics_in_use = 'ABCDEFGH' #ABCDEFGH

max_features = 1000
embedding_dims = 50
maxlen = 50

kfold = 5

will_shuffle = True
shuffle_seed = 1234

n_est = 1000

In [6]:
prf = Profiler.Milestones()

[Profiler starting...]


In [7]:
print("Creating ensemble")
ensemble = ISee.Ensemble()
prf.add_milestone("Created ensemble object")

Creating ensemble


[Profiler] @ 18:26:42: Created ensemble object


In [8]:
print("Setting encoder & tokenizer")
ensemble.set_encoder(LabelEncoder())
ensemble.set_tokenizer(Tokenizer(max_features))
prf.add_milestone("Set encoder & tokenizer")

Setting encoder & tokenizer


[Profiler] @ 18:26:42: Set encoder & tokenizer


In [9]:
print("Reading file")
text_position = 4
class_position = 5
file_name = "brexit_blog_corpus.csv"
with open("datasets/" + file_name) as f:
    lines = [line.rstrip('\n').split('\t') for line in f][1:]
    print("Read {} lines".format(len(lines)))
    print("Applying heuristics " + heuristics_in_use + " (" + str(len(heuristics_in_use)) + ")")
    for h in heuristics_in_use:
        lines = multisub(heuristics[h], lines, text_position)
    line_no = 0
    for line in lines:
        line.append(line_no)
        line_no += 1
prf.add_milestone("Read file")

#print(str(lines[15]))
textos  = [str(a[text_position])  for a in lines]
classes = [str(b[class_position]) for b in lines]
n_classes = len(set(classes))
indexes = [c[-1] for c in lines]
prf.add_milestone("Aggregated data")

Reading file
Read 1682 lines
Applying heuristics ABCDEFGH (8)


[Profiler] @ 18:26:42: Read file
[Profiler] @ 18:26:42: Aggregated data


In [10]:
print("Setting data")
ensemble.set_datasets(textos, classes, indexes)
prf.add_milestone("Set datasets")

Setting data


[Profiler] @ 18:26:43: Set datasets


In [11]:
print("Fitting encoder & tokenizer")
ensemble.fit_encoder()
ensemble.fit_tokenizer()
prf.add_milestone("Fitted encoder & tokenizer")

Fitting encoder & tokenizer


[Profiler] @ 18:26:43: Fitted encoder & tokenizer


In [12]:
print("Splitting data")
ensemble.split_data(shuffle_seed=1337, purge_duplicates=False)
prf.add_milestone("Split data")

Splitting data


[Profiler] @ 18:26:43: Split data


In [13]:
print("Adding models")
##ensemble.add('Random', NullHypothesis.RandomClassifier())
#ensemble.add('XGB',  XGBClassifier(n_estimators=n_est, learning_rate=0.01, max_depth=6, subsample=0.65, colsample_bytree=0.25, gamma=5))
#ensemble.add('Forest', RandomForestClassifier(n_estimators=90))
#ensemble.add('Naive-B',  GaussianNB())
#ensemble.add('SVM_T', svm.SVC(gamma='scale'))
ensemble.add('RNC_binary_crossentropy_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_binary_crossentropy_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_binary_crossentropy_3', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CF', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_categorical_crossentropy_3', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CT', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
prf.add_milestone("Added models")

Adding models


[Profiler] @ 18:26:45: Added models


In [14]:
print("Training ensemble")
ensemble.train(max_features, maxlen, profiler=prf)
prf.add_milestone("Trained ensemble")

[Profiler] @ 18:26:45: Starting training
[Profiler] @ 18:26:45: Fold 1 - Training model RNC_binary_crossentropy_1


Training ensemble
Training with the 1-st set


[Profiler] @ 18:26:58: Fold 1 - Training model RNC_binary_crossentropy_2
[Profiler] @ 18:27:10: Fold 1 - Training model RNC_binary_crossentropy_3
[Profiler] @ 18:27:23: Fold 1 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 18:27:34: Fold 1 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 18:27:46: Fold 1 - Training model RNC_categorical_crossentropy_3


Training with the 2-nd set


[Profiler] @ 18:27:58: Fold 2 - Training model RNC_binary_crossentropy_1
[Profiler] @ 18:28:10: Fold 2 - Training model RNC_binary_crossentropy_2
[Profiler] @ 18:28:21: Fold 2 - Training model RNC_binary_crossentropy_3
[Profiler] @ 18:28:33: Fold 2 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 18:28:44: Fold 2 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 18:28:56: Fold 2 - Training model RNC_categorical_crossentropy_3


...

[Profiler] @ 18:30:17: Fold 4 - Training model RNC_binary_crossentropy_1
[Profiler] @ 18:30:28: Fold 4 - Training model RNC_binary_crossentropy_2
[Profiler] @ 18:30:40: Fold 4 - Training model RNC_binary_crossentropy_3
[Profiler] @ 18:30:51: Fold 4 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 18:31:03: Fold 4 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 18:31:14: Fold 4 - Training model RNC_categorical_crossentropy_3


Training with the 5-th set


[Profiler] @ 18:31:26: Fold 5 - Training model RNC_binary_crossentropy_1
[Profiler] @ 18:31:38: Fold 5 - Training model RNC_binary_crossentropy_2
[Profiler] @ 18:31:49: Fold 5 - Training model RNC_binary_crossentropy_3
[Profiler] @ 18:32:01: Fold 5 - Training model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:13: Fold 5 - Training model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:25: Fold 5 - Training model RNC_categorical_crossentropy_3


Finished training


[Profiler] @ 18:32:37: Trained ensemble


In [15]:
with open('eval_results.txt', 'a') as f:
    f.write(ensemble.evalHistories())

1-st fold
	Model RNC_binary_crossentropy_1
		Max  acc: 90.95%
		Mean acc: 90.14%
		Max  val_acc: 90.00%
		Mean val_acc: 89.85%
		Max  loss: 36.11%
		Mean loss: 28.69%
		Max  val_loss: 31.71%
		Mean val_loss: 29.07%
	Model RNC_binary_crossentropy_2
		Max  acc: 90.66%
		Mean acc: 90.03%
		Max  val_acc: 90.24%
		Mean val_acc: 90.04%
		Max  loss: 37.41%
		Mean loss: 28.85%
		Max  val_loss: 31.05%
		Mean val_loss: 28.70%
	Model RNC_binary_crossentropy_3
		Max  acc: 90.76%
		Mean acc: 90.12%
		Max  val_acc: 90.00%
		Mean val_acc: 89.87%
		Max  loss: 36.15%
		Mean loss: 28.70%
		Max  val_loss: 31.81%
		Mean val_loss: 29.30%
	Model RNC_categorical_crossentropy_1
		Max  acc: 48.40%
		Mean acc: 31.31%
		Max  val_acc: 36.80%
		Mean val_acc: 27.06%
		Max  loss: 214.95%
		Mean loss: 184.74%
		Max  val_loss: 208.77%
		Mean val_loss: 194.11%
	Model RNC_categorical_crossentropy_2
		Max  acc: 48.10%
		Mean acc: 31.90%
		Max  val_acc: 40.06%
		Mean val_acc: 28.72%
		Max  loss: 215.09%
		Mean loss: 184.2

In [16]:
print("Testing ensemble")
res = ensemble.test(max_features, maxlen, profiler=prf)
prf.add_milestone("Tested ensemble")

Testing ensemble
Testing with the 1-st set


[Profiler] @ 18:32:37: Starting testing
[Profiler] @ 18:32:37: Fold 1 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 18:32:38: Fold 1 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 18:32:38: Fold 1 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 18:32:38: Fold 1 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 18:32:38: Fold 1 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 18:32:38: Fold 1 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 18:32:38: Fold 1 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:38: Fold 1 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:38: Fold 1 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:39: Fold 1 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:39: Fold 1 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:39: Fold 1 - Tested model RNC_categorical_crossentropy_3


Testing with the 2-nd set


[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_binary_crossentropy_3
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:39: Fold 2 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:39: Fold 2 - Tested model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:39: Fold 3 - Testing model RNC_binary_crossentropy_1


Testing with the 3-rd set


...

[Profiler] @ 18:32:40: Fold 4 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 18:32:40: Fold 4 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:40: Fold 4 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:40: Fold 4 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:40: Fold 4 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:40: Fold 4 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:40: Fold 4 - Tested model RNC_categorical_crossentropy_3


Testing with the 5-th set


[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_binary_crossentropy_1
[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_binary_crossentropy_1
[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_binary_crossentropy_2
[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_binary_crossentropy_2
[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_binary_crossentropy_3


Finished testing


[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_binary_crossentropy_3
[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_categorical_crossentropy_1
[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_categorical_crossentropy_2
[Profiler] @ 18:32:40: Fold 5 - Testing model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:40: Fold 5 - Tested model RNC_categorical_crossentropy_3
[Profiler] @ 18:32:40: Tested ensemble


In [17]:
ensemble.evaluate(verbose=True)
prf.add_milestone("Finished evaluating")

Evaluating 1-th fold
	     -  Correct - RNC_binary_crossentropy_1 - RNC_binary_crossentropy_2 - RNC_binary_crossentropy_3 - RNC_categorical_crossentropy_1 - RNC_categorical_crossentropy_2 - RNC_categorical_crossentropy_3 -   Major.- Phrase
	   0 - agreement/disagreement - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* i know what you mean.
	   1 - agreement/disagreement - agreement/disagreement* - contrariety  - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - contrariety  - agreement/disagreement* i don't disagree that yes campaign made mistakes, but they must be contextualised within truly immense opposition they had.
	   2 - agreement/disagreement - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreement* - agreement/disagreemen

	 119 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* had ecb started broad bond-buying programme in early 2009, before sovereign debt crisis was on horizon, yields might never have spun out of control as they did.
	 120 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* the political elites new all along that had eu project been presented to people for what it really is, people in all countries would have rejected it.
	 121 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* his position is precarious, and defeat would have badly damaged his government.
	 122 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality*

	 248 - source of knowledge - uncertainty  - uncertainty  - uncertainty  - source of knowledge* - source of knowledge* - uncertainty  - uncertainty  deflationary risks may have peaked, it said; companies’ cash flows look healthy, as does job market.
	 249 - source of knowledge - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* that may not_ government-currently fighting to raise inflation-too much this time; many analysts expect slower second quarter following bumper first.
	 250 - source of knowledge - source of knowledge* - contrariety  - contrariety  - source of knowledge* - source of knowledge* - contrariety  - source of knowledge* roosevelt saw world as quasi-unipolar one in which _laugh_ controlled international politics, nominally within united nations framework, together with china, russia and uk, but in reality as unquestionably dominant actor.
	 251 - source of knowl

	  52 - contrariety - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* so, even people who have lived here for 20 years, and are married to brits, won't be allowed say in their future.
	  53 - contrariety - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* everywhere constitutions are in crisis - even to some in britain.
	  54 - contrariety - contrariety* - contrariety* - contrariety* - source of knowledge  - source of knowledge  - contrariety* - contrariety* european lenders' claims on greece, which once topped euro 300 billion, now stand at just euro 54 billion, according to deutsche bank.
	  55 - contrariety - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* - contrariety* in spite of their aspirations scotland _laugh_ no indigenous wind turbine manufacturing or technology.
	  56 - contrariety - contrariety* - contrariety* -  necessity  - 

	 193 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* so, even people who have lived here for 20 years, and are married to brits, won’t be allowed say in their future.
	 194 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* once they get hang of voting, they will keep voting.
	 195 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* the 1975 referendum on whether to remain in then eec was expected to be close.
	 196 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* back in real world, greece is running out of money.
	 197 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* i was very happy and peaceful during my stay here and i will back for sure.
	 198 - prediction - uncertainty  - uncer

...

	 183 - prediction - prediction* - prediction* - prediction* - source of knowledge  - prediction* - prediction* - prediction* once new government takes power, united kingdom looks set to gain firm ally.
	 184 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* jewish king jesus is coming at rapture for us in clouds-don't miss it for world.
	 185 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* realistically, best they can hope for is one msp, taken from snp.
	 186 - prediction - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* - prediction* the bt campaign will be reduced to pro union song written by david bowie
	 187 - prediction - prediction* - contrariety  - prediction* - contrariety  - prediction* - contrariety  - prediction* they'll want to savour this - not_ both feet off 5 minutes after returning to power.
	 188 - prediction -

	 312 - uncertainty - prediction  - uncertainty* - prediction  - uncertainty* - prediction  - uncertainty* - prediction  the added complication is likely to make more of these people unlikely to vote at all.
	 313 - uncertainty - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* as consequence, chinese banks could, at government's behest, offset collapse of external demand by flooding economy with credit.
	 314 - uncertainty - source of knowledge  - uncertainty* - uncertainty* - uncertainty* - prediction  - uncertainty* - uncertainty* ‘that such an unnecessary and irrational project as building european super-state was ever embarked on,’ wrote lady thatcher, ‘will seem in future years to be perhaps greatest folly of modern era.
	 315 - uncertainty - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* - uncertainty* and maybe he would come inside and we could talk.
	 316 - uncertainty -  necessity  

	  98 - hypotheticality - uncertainty  - contrariety  - contrariety  - contrariety  - contrariety  - contrariety  - contrariety  and so, despite brinkmanship of past weeks, even if europe’s leaders hammer out an accord with greece they may simply buy more time rather than remove risk of grexit.
	  99 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* a month earlier and i would have been singaporean, 3 months later and i would have been german.
	 100 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* outside eu, we’d still be free to trade with europe.
	 101 - hypotheticality - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* - hypotheticality* relations with devious mr tsipras are shattered: with him in charge, they will strug

	 220 - source of knowledge - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* its also worth remembering that much of heathrow is given over to cargo.
	 221 - source of knowledge - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* in his speech, duda called for an increased nato presence in region and said an adjustment to polish foreign policy was required.
	 222 - source of knowledge - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* records show attorney general’s investigation began at least as early as 2010.
	 223 - source of knowledge - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source of knowledge* - source

[Profiler] @ 18:32:43: Finished evaluating


In [18]:
print("Profiler data:")
prf.exhibit(max_d=10)

Profiler data:
0=1=2=3=4=5=6=7=8=9=10==========11========12=========13========14========15=========16========17========18========19========20========21=========22========23========24========25========26========27========28========29========30========31========32========33========34========35========36========37========38========39=========40=41=42=43=44=45=46=47=48=49=50=51=52=53=54=55=56=57=58=59=60=61=62=63=64=65=66=67=68=69=70=71=72=73=74=75=76=77=78=79=80=81=82=83=84=85=86=87=88=89=90=91=92=93=94=95=96=97=98=99=100=101=102==103
  0: Start - 19/05/2019 - 18:26:42
  1: Created ensemble object - 19/05/2019 - 18:26:42 - 0.12s
  2: Set encoder & tokenizer - 19/05/2019 - 18:26:42 - 0.10s
  3: Read file - 19/05/2019 - 18:26:42 - 0.39s
  4: Aggregated data - 19/05/2019 - 18:26:42 - 0.00s
  5: Set datasets - 19/05/2019 - 18:26:43 - 0.01s
  6: Fitted encoder & tokenizer - 19/05/2019 - 18:26:43 - 0.62s
  7: Split data - 19/05/2019 - 18:26:43 - 0.14s
  8: Added models - 19/05/2019 - 18:26:45 -